In [ ]:
import requests
from spot.utils import proj_root

# Replace `example.py` with your desired Python source file.
with open(proj_root() / 'data/code/good_code_2.py') as f:
    r = requests.post("https://type4py.com/api/predict?tc=0", f.read())
r.json()

In [ ]:
res_json = r.json()["response"]
res_json

In [13]:
res_json["classes"][0]["variables_p"]

{'ctx_args': [['str', 0.262955513851708],
  ['int', 0.20040486741829183],
  ['Dict[str, Dict[str, Any]]', 0.14470794278737809],
  ['List[Union[Any, Any]]', 0.1076166887940718],
  ['List[str]', 0.10089842318062654],
  ['list', 0.07659333086864245],
  ['Type[Tuple[Any, Any]]', 0.05063646266431431]],
 'diversity_penalty': [['str', 0.4],
  ['List[str]', 0.1],
  ['int', 0.1],
  ['raiden.utils.WithdrawAmount', 0.1],
  ['Dict[str, List[Any]]', 0.1],
  ['Tuple[str, str, str, str, str, str, str, str]', 0.1],
  ['numpy.array', 0.1]],
 'do_sample': [['bool', 0.49999999999999994],
  ['int', 0.3],
  ['str', 0.19999999999999998]],
 'length_penalty': [['str', 0.4],
  ['List[str]', 0.1],
  ['int', 0.1],
  ['raiden.utils.WithdrawAmount', 0.1],
  ['Dict[str, List[Any]]', 0.1],
  ['Tuple[str, str, str, str, str, str, str, str]', 0.1],
  ['numpy.array', 0.1]],
 'max_workers': [['str', 0.4],
  ['List[str]', 0.1],
  ['int', 0.1],
  ['raiden.utils.WithdrawAmount', 0.1],
  ['Dict[str, List[Any]]', 0.1],
  ['T